In [2]:
from datetime import datetime, date
import pandas as pd
import plotly.express as px

from pyspark.ml import Pipeline
from pyspark.sql import Row
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler

### Modelos utilizados ###

from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier

from pyspark.sql import Row
from pyspark.sql import SQLContext

spark = SparkSession \
    .builder \
    .appName("ML Pipeline") \
    .getOrCreate()

sc = SparkContext.getOrCreate();

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/06 20:49:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/06 20:49:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/07/06 20:49:39 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [14]:
url = "https://raw.githubusercontent.com/Meliande/lab-ia/main/Data/preprocessado/Farol200/out.csv"
from pyspark import SparkFiles
sc.addFile(url)
sqlContext = SQLContext(sc)

Farol200 = sqlContext.read.csv(SparkFiles.get("out.csv"), header=True, inferSchema= True)
Farol200 = Farol200.drop("_c0")
Farol200.toPandas()

23/07/06 20:58:03 WARN SparkContext: The path https://raw.githubusercontent.com/Meliande/lab-ia/main/Data/preprocessado/Farol200/out.csv has been added already. Overwriting of added paths is not supported in the current version.
/home/rodrigo/.local/lib/python3.11/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


,Inicio_Semana,Total de Chuvas(mm),Média diária de chuvas(mm),categoria,ponto_codigo,Categoria_Indexada
0,2008-08-10,17.4,2.485714,Indisponível,Farol da Barra - SSA FB 200,2.0
1,2007-08-05,22.4,3.200000,Indisponível,Farol da Barra - SSA FB 200,2.0
2,2008-05-11,5.8,0.828571,Indisponível,Farol da Barra - SSA FB 200,2.0
3,2007-03-11,53.0,7.571429,Indisponível,Farol da Barra - SSA FB 200,2.0
4,2007-07-08,17.0,2.428571,Própria,Farol da Barra - SSA FB 200,0.0
...,...,...,...,...,...,...
653,2022-01-02,9.6,1.371429,Própria,Farol da Barra - SSA FB 200,0.0
654,2022-04-03,32.0,4.571429,Indisponível,Farol da Barra - SSA FB 200,2.0
655,2023-02-26,17.0,2.428571,Imprópria,Farol da Barra - SSA FB 200,1.0
656,2022-10-02,0.2,0.028571,Própria,Farol da Barra - SSA FB 200,0.0


In [16]:
Farol200 = Farol200.drop('categoria')

Farol200 = Farol200.filter(Farol200.Categoria_Indexada < 2)

Farol200.toPandas()

,Inicio_Semana,Total de Chuvas(mm),Média diária de chuvas(mm),ponto_codigo,Categoria_Indexada
0,2007-07-08,17.0,2.428571,Farol da Barra - SSA FB 200,0.0
1,2008-06-01,0.2,0.028571,Farol da Barra - SSA FB 200,0.0
2,2008-06-22,58.8,8.400000,Farol da Barra - SSA FB 200,0.0
3,2008-07-20,25.4,3.628571,Farol da Barra - SSA FB 200,0.0
4,2008-08-17,35.2,5.028572,Farol da Barra - SSA FB 200,0.0
...,...,...,...,...,...
554,2023-04-23,94.8,13.542858,Farol da Barra - SSA FB 200,1.0
555,2023-01-08,3.8,0.542857,Farol da Barra - SSA FB 200,1.0
556,2022-01-02,9.6,1.371429,Farol da Barra - SSA FB 200,0.0
557,2023-02-26,17.0,2.428571,Farol da Barra - SSA FB 200,1.0


In [17]:
informacoes_necessarias = ['Total de Chuvas(mm)','Média diária de chuvas(mm)']

assembler = VectorAssembler(inputCols=informacoes_necessarias, outputCol='informacoes')

dfFarol200 = assembler.transform(Farol200)

dfFarol200.toPandas()

,Inicio_Semana,Total de Chuvas(mm),Média diária de chuvas(mm),ponto_codigo,Categoria_Indexada,informacoes
0,2007-07-08,17.0,2.428571,Farol da Barra - SSA FB 200,0.0,"[17.0, 2.4285714285714284]"
1,2008-06-01,0.2,0.028571,Farol da Barra - SSA FB 200,0.0,"[0.2, 0.028571428997176036]"
2,2008-06-22,58.8,8.400000,Farol da Barra - SSA FB 200,0.0,"[58.8, 8.39999989100865]"
3,2008-07-20,25.4,3.628571,Farol da Barra - SSA FB 200,0.0,"[25.4, 3.6285713740757535]"
4,2008-08-17,35.2,5.028572,Farol da Barra - SSA FB 200,0.0,"[35.2, 5.028571537562779]"
...,...,...,...,...,...,...
554,2023-04-23,94.8,13.542858,Farol da Barra - SSA FB 200,1.0,"[94.8, 13.542857578822545]"
555,2023-01-08,3.8,0.542857,Farol da Barra - SSA FB 200,1.0,"[3.8, 0.5428571360451835]"
556,2022-01-02,9.6,1.371429,Farol da Barra - SSA FB 200,0.0,"[9.6, 1.3714286259242467]"
557,2023-02-26,17.0,2.428571,Farol da Barra - SSA FB 200,1.0,"[17.0, 2.4285714285714284]"


In [18]:
### Dividindo o dataser em treino (80%) e teste (20%) ###

(treinoFarol200, testeFarol200) = dfFarol200.randomSplit([0.8,0.2])

treinoFarol200.toPandas()

,Inicio_Semana,Total de Chuvas(mm),Média diária de chuvas(mm),ponto_codigo,Categoria_Indexada,informacoes
0,2007-02-04,61.6,8.800000,Farol da Barra - SSA FB 200,0.0,"[61.6, 8.799999782017299]"
1,2007-02-11,76.2,10.885714,Farol da Barra - SSA FB 200,0.0,"[76.2, 10.885713849748884]"
2,2007-07-01,44.4,6.342857,Farol da Barra - SSA FB 200,0.0,"[44.4, 6.342857360839844]"
3,2007-07-08,17.0,2.428571,Farol da Barra - SSA FB 200,0.0,"[17.0, 2.4285714285714284]"
4,2008-05-18,26.0,3.714286,Farol da Barra - SSA FB 200,0.0,"[26.0, 3.7142857142857144]"
...,...,...,...,...,...,...
441,2023-03-26,36.2,5.171429,Farol da Barra - SSA FB 200,1.0,"[36.2, 5.171428680419922]"
442,2023-04-02,66.4,9.485715,Farol da Barra - SSA FB 200,1.0,"[66.4, 9.485714503696986]"
443,2023-04-16,6.6,0.942857,Farol da Barra - SSA FB 200,1.0,"[6.6, 0.9428571292332241]"
444,2023-04-23,94.8,13.542858,Farol da Barra - SSA FB 200,1.0,"[94.8, 13.542857578822545]"


In [19]:
### Definindo os modelos ###

gbt = GBTClassifier(labelCol="Categoria_Indexada", featuresCol="informacoes", maxIter=10)
dt = DecisionTreeClassifier(labelCol='Categoria_Indexada',featuresCol='informacoes')
rf = RandomForestClassifier(labelCol='Categoria_Indexada',featuresCol='informacoes',maxDepth=5)

In [20]:
### Treinando os modelos #### 

gbtModel = gbt.fit(treinoFarol200) ### Gradient Boosted Tree Classifier

rfModel = rf.fit(treinoFarol200) ### Random Forest Classifier

dtModel = dt.fit(treinoFarol200) ### Decision Tree classfier

rfModel.write().overwrite().save('models/rf')

gbtModel.write().overwrite().save('models/gbt')

dtModel.write().overwrite().save('models/dt')

In [21]:
### Testando os modelos ###

gbtPredicao = gbtModel.transform(testeFarol200)

gbtPredicao.toPandas()

23/07/06 20:58:37 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/07/06 20:58:37 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


,Inicio_Semana,Total de Chuvas(mm),Média diária de chuvas(mm),ponto_codigo,Categoria_Indexada,informacoes,rawPrediction,probability,prediction
0,2007-07-15,3.2,0.457143,Farol da Barra - SSA FB 200,0.0,"[3.2, 0.45714286395481657]","[0.12424009835075671, -0.12424009835075671]","[0.5618023902150125, 0.43819760978498745]",0.0
1,2007-07-22,24.0,3.428571,Farol da Barra - SSA FB 200,0.0,"[24.0, 3.4285714285714284]","[0.5622426410485505, -0.5622426410485505]","[0.754819742115896, 0.24518025788410402]",0.0
2,2008-06-29,15.8,2.257143,Farol da Barra - SSA FB 200,0.0,"[15.8, 2.257142884390695]","[0.5517051333115796, -0.5517051333115796]","[0.7508985413474322, 0.24910145865256783]",0.0
3,2008-07-13,29.6,4.228571,Farol da Barra - SSA FB 200,0.0,"[29.6, 4.228571483067104]","[0.7380668168727264, -0.7380668168727264]","[0.8139878785115133, 0.18601212148848667]",0.0
4,2008-08-17,35.2,5.028572,Farol da Barra - SSA FB 200,0.0,"[35.2, 5.028571537562779]","[0.45235253115844076, -0.45235253115844076]","[0.7119154338789533, 0.2880845661210467]",0.0
...,...,...,...,...,...,...,...,...,...
108,2023-03-12,57.6,8.228571,Farol da Barra - SSA FB 200,0.0,"[57.6, 8.228571210588727]","[0.5301942714264468, -0.5301942714264468]","[0.7427647893448505, 0.25723521065514954]",0.0
109,2023-03-19,48.4,6.914286,Farol da Barra - SSA FB 200,1.0,"[48.4, 6.914285932268415]","[0.7225019637341391, -0.7225019637341391]","[0.8092283424748533, 0.1907716575251467]",0.0
110,2023-04-09,12.8,1.828571,Farol da Barra - SSA FB 200,1.0,"[12.8, 1.8285714558192663]","[0.3841824610361759, -0.3841824610361759]","[0.68316708745748, 0.31683291254252]",0.0
111,2023-04-30,49.4,7.057143,Farol da Barra - SSA FB 200,1.0,"[49.4, 7.057143075125558]","[0.7225019637341391, -0.7225019637341391]","[0.8092283424748533, 0.1907716575251467]",0.0


In [23]:
rfPredicao = rfModel.transform(testeFarol200)

rfPredicao.toPandas()

,Inicio_Semana,Total de Chuvas(mm),Média diária de chuvas(mm),ponto_codigo,Categoria_Indexada,informacoes,rawPrediction,probability,prediction
0,2007-07-15,3.2,0.457143,Farol da Barra - SSA FB 200,0.0,"[3.2, 0.45714286395481657]","[13.65564378789206, 6.344356212107941]","[0.682782189394603, 0.31721781060539705]",0.0
1,2007-07-22,24.0,3.428571,Farol da Barra - SSA FB 200,0.0,"[24.0, 3.4285714285714284]","[14.022263466433447, 5.977736533566552]","[0.7011131733216723, 0.2988868266783276]",0.0
2,2008-06-29,15.8,2.257143,Farol da Barra - SSA FB 200,0.0,"[15.8, 2.257142884390695]","[14.341496767515157, 5.658503232484844]","[0.7170748383757578, 0.2829251616242422]",0.0
3,2008-07-13,29.6,4.228571,Farol da Barra - SSA FB 200,0.0,"[29.6, 4.228571483067104]","[14.270167321428792, 5.729832678571208]","[0.7135083660714396, 0.2864916339285604]",0.0
4,2008-08-17,35.2,5.028572,Farol da Barra - SSA FB 200,0.0,"[35.2, 5.028571537562779]","[14.083600157249688, 5.916399842750312]","[0.7041800078624844, 0.29581999213751564]",0.0
...,...,...,...,...,...,...,...,...,...
108,2023-03-12,57.6,8.228571,Farol da Barra - SSA FB 200,0.0,"[57.6, 8.228571210588727]","[14.448154673919824, 5.551845326080172]","[0.7224077336959913, 0.27759226630400863]",0.0
109,2023-03-19,48.4,6.914286,Farol da Barra - SSA FB 200,1.0,"[48.4, 6.914285932268415]","[14.448154673919824, 5.551845326080172]","[0.7224077336959913, 0.27759226630400863]",0.0
110,2023-04-09,12.8,1.828571,Farol da Barra - SSA FB 200,1.0,"[12.8, 1.8285714558192663]","[14.651833884720258, 5.348166115279743]","[0.7325916942360129, 0.26740830576398716]",0.0
111,2023-04-30,49.4,7.057143,Farol da Barra - SSA FB 200,1.0,"[49.4, 7.057143075125558]","[14.448154673919824, 5.551845326080172]","[0.7224077336959913, 0.27759226630400863]",0.0


In [22]:
dtPredicao = dtModel.transform(testeFarol200)

dtPredicao.toPandas()

,Inicio_Semana,Total de Chuvas(mm),Média diária de chuvas(mm),ponto_codigo,Categoria_Indexada,informacoes,rawPrediction,probability,prediction
0,2007-07-15,3.2,0.457143,Farol da Barra - SSA FB 200,0.0,"[3.2, 0.45714286395481657]","[314.0, 132.0]","[0.7040358744394619, 0.29596412556053814]",0.0
1,2007-07-22,24.0,3.428571,Farol da Barra - SSA FB 200,0.0,"[24.0, 3.4285714285714284]","[314.0, 132.0]","[0.7040358744394619, 0.29596412556053814]",0.0
2,2008-06-29,15.8,2.257143,Farol da Barra - SSA FB 200,0.0,"[15.8, 2.257142884390695]","[314.0, 132.0]","[0.7040358744394619, 0.29596412556053814]",0.0
3,2008-07-13,29.6,4.228571,Farol da Barra - SSA FB 200,0.0,"[29.6, 4.228571483067104]","[314.0, 132.0]","[0.7040358744394619, 0.29596412556053814]",0.0
4,2008-08-17,35.2,5.028572,Farol da Barra - SSA FB 200,0.0,"[35.2, 5.028571537562779]","[314.0, 132.0]","[0.7040358744394619, 0.29596412556053814]",0.0
...,...,...,...,...,...,...,...,...,...
108,2023-03-12,57.6,8.228571,Farol da Barra - SSA FB 200,0.0,"[57.6, 8.228571210588727]","[314.0, 132.0]","[0.7040358744394619, 0.29596412556053814]",0.0
109,2023-03-19,48.4,6.914286,Farol da Barra - SSA FB 200,1.0,"[48.4, 6.914285932268415]","[314.0, 132.0]","[0.7040358744394619, 0.29596412556053814]",0.0
110,2023-04-09,12.8,1.828571,Farol da Barra - SSA FB 200,1.0,"[12.8, 1.8285714558192663]","[314.0, 132.0]","[0.7040358744394619, 0.29596412556053814]",0.0
111,2023-04-30,49.4,7.057143,Farol da Barra - SSA FB 200,1.0,"[49.4, 7.057143075125558]","[314.0, 132.0]","[0.7040358744394619, 0.29596412556053814]",0.0


In [ ]:
### Avaliando os modelos ### 

### Definindo os avaliadores ###

### Documentação => https://spark.apache.org/docs/latest/mllib-evaluation-metrics.html ###

### https://towardsdatascience.com/the-f1-score-bec2bbc38aa6 ###

acuracia = MulticlassClassificationEvaluator(labelCol='Categoria_Indexada',predictionCol='prediction',metricName='accuracy')
f1 = MulticlassClassificationEvaluator(labelCol='Categoria_Indexada',predictionCol='prediction',metricName='f1')
precisaoPonderada = MulticlassClassificationEvaluator(labelCol='Categoria_Indexada',predictionCol='prediction',metricName='weightedPrecision')
weightedRecall = MulticlassClassificationEvaluator(labelCol='Categoria_Indexada',predictionCol='prediction',metricName='weightedRecall')

### Resultados da Acurácia ###

gbtAcuracia = acuracia.evaluate(gbtPredicao)
rfAcuracia = acuracia.evaluate(rfPredicao)
dtAcuracia = acuracia.evaluate(dtPredicao)
print('Acurácia do teste Árvore de Decisão (Gradiente Boosting) = ', gbtAcuracia)
print('Acurácia do teste Árvore Aleatória = ', rfAcuracia)
print('Acurácia do teste Árvore de Decisão = ', dtAcuracia)

In [ ]:
### Resultados do F1 ###

gbtF1 = f1.evaluate(gbtPredicao)
rfF1 = f1.evaluate(rfPredicao)
dtF1 = f1.evaluate(dtPredicao)
print('F1 do teste Árvore de Decisão (Gradiente Boosting) = ', gbtF1)
print('F1 do teste Árvore Aleatória = ', rfF1)
print('F1 do teste Árvore de Decisão = ', dtF1)

In [ ]:
### Resultados do Precisão Ponderada ###

gbtPP = precisaoPonderada.evaluate(gbtPredicao)
rfPP = precisaoPonderada.evaluate(rfPredicao)
dtPP = precisaoPonderada.evaluate(dtPredicao)
print('Precisão Ponderada do teste Árvore de Decisão (Gradiente Boosting) = ', gbtPP)
print('Precisão Ponderada do teste Árvore Aleatória = ', rfPP)
print('Precisão Ponderada do teste Árvore de Decisão = ', dtPP)

In [ ]:
### Resultados do weightedRecall ###

### Recall é a razão entre o número de positivos verdadeiros (pv) e a soma dos positivos verdadeiros (pv) e falsos negativos (fn) => pv/(pv+fn)

gbtWR = weightedRecall.evaluate(gbtPredicao)
rfWR = weightedRecall.evaluate(rfPredicao)
dtWR = weightedRecall.evaluate(dtPredicao)
print('Recall do teste Árvore de Decisão (Gradiente Boosting) = ', gbtWR)
print('Recall do teste Árvore Aleatória = ', rfWR)
print('Recall do teste Árvore de Decisão = ', dtWR)